In [5]:
import os
import cv2
import pprint
import rasterio
import numpy as np
from PIL import Image
from pathlib import Path
from rasterio.mask import mask
from rasterio.plot import reshape_as_image
from shapely.geometry import Polygon, mapping
from preprocess import rotateAndDeleteEmptySpace

In [ ]:
def getMainData(mainDataPath, dataType):
    '''
    get all data path 
    '''
    filePath = []

    # only get the data path from selected data type
    mainTargetDataPath = mainDataPath + '/' + dataType

    # check if the data type exist
    if(not(os.path.exists(mainTargetDataPath))):
        print(mainTargetDataPath, "path not exist")
        return filePath
    
    # loop into each main picture (day)
    for folder in os.listdir(mainTargetDataPath):
        folderPath = mainTargetDataPath + "/" + folder

        # loop into each label
        for subFolder in os.listdir(folderPath):
            subFolderPath = folderPath + "/" + subFolder

            # loop into each file
            for targetFile in os.listdir(subFolderPath):
                targetFilePath = subFolderPath + "/" + targetFile
                filePath.append(targetFilePath)
    
    return(filePath)

def checkFileType(pathList, fileType):
    '''
    check file type in the data 
    get only the file type
    '''
    trueFile = []
    for file in pathList:
        
        # check last n character if same as selected file type
        if(file[-len(fileType):] == fileType):
            trueFile.append(file)
    return trueFile

def loopRotateTifFile(targetTifFile, rotationDegree):
    '''
    loop all the file, rotate tif file and return new file path as list
    '''
    newTifFilePath = []
    notrotate = []
    count = 1

    # loop into every target file
    for tifFile in targetTifFile:

        # create new file name (original: D:/.../RGB_202401181250_1_original.tif) to (new:D:/.../RGB_202401181250_1_originalFixTilt.tif)
        newFileName = tifFile[0:-4] + "FixTilt" + ".tif"

        # check if file already exist
        if(not(os.path.exists(newFileName))):

            # call rotate function 
            try:
                rotateAndDeleteEmptySpace(tifFile, newFileName, rotationDegree)
            except:
                notrotate.append(newFileName)
    
        else:
            print("file already exist")
        
        # add new file name to list
        newTifFilePath.append(newFileName)
        print(count, '/', len(targetTifFile))
        count += 1
        
    return newTifFilePath, notrotate

def convert_tiff_to_jpg(input_tif, output_jpg):
    """
    Converts an RGB TIFF image to JPG.

    Parameters:
    - input_tif: Path to input TIFF image.
    - output_jpg: Path to save output JPG image.
    """
    with rasterio.open(input_tif) as src:
        image = reshape_as_image(src.read())  # Convert to NumPy array

    # Convert RGB to BGR for OpenCV compatibility
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Save as JPG with high quality
    cv2.imwrite(output_jpg, image, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

def loopConvertTifToJpgFile(targetTifFile):
    '''
    loop all the file, convert tif file to jpg
    '''
    newJpgFilePath = []
    notConvert = []
    count = 1

    # loop into every target file
    for tifFile in targetTifFile:

        # create new file name (original: D:/.../RGB_202401181250_1_original.tif) to (new:D:/.../RGB_202401181250_1_originalFixTilt.tif)
        newFileName = tifFile[:-11] + ".jpg"

        # check if file already exist
        if(not(os.path.exists(newFileName))):

            # call rotate function 
            try:
                print(newFileName)
                convert_tiff_to_jpg(tifFile, newFileName)
            except:
                notConvert.append(newFileName)
    
        else:
            print("file already exist")
        
        # add new file name to list
        newJpgFilePath.append(newFileName)
        print(count, '/', len(targetTifFile))
        count += 1
        
    return newJpgFilePath, notConvert


# def rotateAndDeleteEmptySpaceXX(fileName, outputName, degreeOfRotation):
#     """
#     Rotates a multi-band raster image, removes empty space, and saves the result.

#     Parameters:
#     - fileName: Path to input TIFF file.
#     - outputName: Path to save the rotated and cropped TIFF file.
#     - degreeOfRotation: Angle in degrees for rotation.
#     """
#     with rasterio.open(fileName) as src:
#         data = src.read()  # Read all bands (shape: bands, height, width)
#         profile = src.profile

#         rotated_bands = []
#         min_row, min_col, max_row, max_col = None, None, None, None

#         # Rotate all bands and determine common crop area
#         for i in range(data.shape[0]):  
#             band = data[i]
#             pil_band = Image.fromarray(band)
#             rotated_band = pil_band.rotate(degreeOfRotation, expand=True)
#             rotated_band_data = np.array(rotated_band)

#             # Find non-empty pixels
#             non_empty_pixels = rotated_band_data > 0
#             rows = np.any(non_empty_pixels, axis=1)
#             cols = np.any(non_empty_pixels, axis=0)

#             # Get min/max row/column indices
#             rmin, rmax = np.where(rows)[0][[0, -1]]
#             cmin, cmax = np.where(cols)[0][[0, -1]]

#             # Update global min/max crop dimensions
#             if min_row is None or rmin > min_row:
#                 min_row = rmin
#             if max_row is None or rmax < max_row:
#                 max_row = rmax
#             if min_col is None or cmin > min_col:
#                 min_col = cmin
#             if max_col is None or cmax < max_col:
#                 max_col = cmax

#             rotated_bands.append(rotated_band_data)

#         # Apply the same crop dimensions to all bands
#         cropped_bands = [band[min_row:max_row+1, min_col:max_col+1] for band in rotated_bands]

#         # Stack cropped bands into a multi-band image
#         cropped_data = np.stack(cropped_bands)

#         # Update profile metadata
#         profile.update(width=cropped_data.shape[2], height=cropped_data.shape[1])

#         # Save the rotated and cropped image
#         with rasterio.open(outputName, 'w', **profile) as dst:
#             dst.write(cropped_data)

#         print(f"✅ Rotated and saved: {outputName}")

In [7]:
dataPath = "/Volumes/PortableSSD/dataForProcess/2025MainData"
RGBDataPath = getMainData(dataPath, "RGB")
targetTifFile = checkFileType(RGBDataPath, "original.tif")
newTifFilePath, notrotate = loopRotateTifFile(targetTifFile, 103)

# print(notrotate)
# rotateAndDeleteEmptySpaceXX('D:/ice-wheat/data/dataForProcess/mainData/RGB/RGB_202402131116/RGB_202402131116_67/RGB_202402131116_67_original.tif', 'D:/ice-wheat/data/dataForProcess/mainData/RGB/RGB_202402131116/RGB_202402131116_67/RGB_202402131116_67_originalFixTilt.tif', 103)
# for file in RGBDataPath:
#     if(file[-11:-4] == "FixTilt"):
#         os.remove(file) 
#         print(file)

file already exist
1 / 961
file already exist
2 / 961
file already exist
3 / 961
file already exist
4 / 961
file already exist
5 / 961
file already exist
6 / 961
file already exist
7 / 961
file already exist
8 / 961
file already exist
9 / 961
file already exist
10 / 961
file already exist
11 / 961
file already exist
12 / 961
file already exist
13 / 961
file already exist
14 / 961
file already exist
15 / 961
file already exist
16 / 961
file already exist
17 / 961
file already exist
18 / 961
file already exist
19 / 961
file already exist
20 / 961
file already exist
21 / 961
file already exist
22 / 961
file already exist
23 / 961
file already exist
24 / 961
file already exist
25 / 961
file already exist
26 / 961
file already exist
27 / 961
file already exist
28 / 961
file already exist
29 / 961
file already exist
30 / 961
file already exist
31 / 961
file already exist
32 / 961
file already exist
33 / 961
file already exist
34 / 961
file already exist
35 / 961
file already exist
36 / 961
f

In [8]:
newJpgFilePath, notConvert = loopConvertTifToJpgFile(newTifFilePath)

/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202504040926/RGB_202504040926_1/RGB_202504040926_1_original.jpg
/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202504040926/RGB_202504040926_2/RGB_202504040926_2_original.jpg
/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202504040926/RGB_202504040926_3/RGB_202504040926_3_original.jpg
/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202504040926/RGB_202504040926_4/RGB_202504040926_4_original.jpg
/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202504040926/RGB_202504040926_5/RGB_202504040926_5_original.jpg
/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202504040926/RGB_202504040926_6/RGB_202504040926_6_original.jpg
/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202504040926/RGB_202504040926_7/RGB_202504040926_7_original.jpg
/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202504040926/RGB_202504040926_8/RGB_202504040926_8_original.jpg
/Volumes/PortableSSD/dataForProc